Para análises mais avançadas, vamos buscar vincular os três conjuntos principais de dados: pavimentos, acidentes e contagem de tráfego.

In [1]:
################################################################################
# Definindo um caminho personalizado para bibliotecas R
################################################################################
# Exemplo de caminho: pode ser uma pasta dentro do seu diretório home
# Lembre-se que o R pode adicionar subdiretórios específicos da versão/arquitetura
meu_caminho_biblioteca <- "~/R/library"

# Define a variável de ambiente R_LIBS_USER para a sessão atual
Sys.setenv(R_LIBS_USER = meu_caminho_biblioteca)

# Verifica o valor da variável de ambiente
Sys.getenv("R_LIBS_USER")

# Mostra todos os caminhos de biblioteca que o R está usando
# O seu novo caminho deve aparecer aqui, geralmente como o primeiro da lista.
.libPaths()

[1] "~/R/library"

[1] "/usr/local/lib/R/site-library" "/usr/local/lib/R/library"     
[3] "/home/vscode/R/library"

In [2]:
################################################################################
#               INSTALAÇÃO E CARREGAMENTO DE PACOTES NECESSÁRIOS               #
################################################################################

# Pacotes utilizados
pacotes <- c("tidyverse", #carregar outros pacotes do R
             "knitr", "kableExtra", #formatação de tabelas
             "arrow","duckdb") # parquet e banco de dados

if(sum(as.numeric(!pacotes %in% installed.packages())) != 0){
  instalador <- pacotes[!pacotes %in% installed.packages()]
  for(i in 1:length(instalador)) {
    install.packages(instalador, dependencies = T, lib = meu_caminho_biblioteca)
    break()}
  sapply(pacotes, require, character = T, lib = meu_caminho_biblioteca) 
} else {
  sapply(pacotes, require, character = T, lib = meu_caminho_biblioteca) 
}

Loading required package: tidyverse



── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.6.0
✔ ggplot2   4.0.1     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.2.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: knitr

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘knitr’”
Loading required package: kableExtra


Attaching package: ‘kableExtra’


The following object is masked from ‘package:dplyr’:

    group_rows


Loading required package: arrow


Attaching package: ‘arrow’


The following object is masked from ‘package:lubridate’:

    duration


The following object is

tidyverse      knitr kableExtra      arrow     duckdb 
      TRUE      FALSE       TRUE       TRUE       TRUE

In [6]:
# Para limpar a memória da sessão R removendo todos os objetos do ambiente global, 
# execute rm(list = ls(all.names = TRUE)). 
# Em seguida, rode gc() para forçar a coleta de lixo e liberar a memória fisicamente alocada.​
rm(list = ls(all.names = TRUE))
gc()
ls()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1654859,88.4,2709718,144.8,2709718,144.8
Vcells,2864183,21.9,13685346,104.5,11713589,89.4


character(0)

In [3]:
load("Trabalho/bases-de-dados/acidentes/datatran2024.RData")
load("Trabalho/bases-de-dados/condicoes-de-pavimento/mes-a-mes/icm-mes-a-mes.Rdata")
load("Trabalho/bases-de-dados/contagem-de-trafego/contagem-de-trafego.RData")
load("Trabalho/bases-de-dados/definicao-de-pavimento/df_snv.RData")
ls()

[1] "df_acidentes_prf"       "df_snv"                 "df_trafego"            
[4] "icm_mes_a_mes"          "meu_caminho_biblioteca" "pacotes"

In [2]:
head(df_acidentes_prf)
head(icm_mes_a_mes)
head(df_trafego)
head(df_snv)

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,⋯,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop
,<int>,<date>,<chr>,<hms>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,⋯,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>
1,571789,2024-01-01,segunda-feira,14160 secs,ES,101,38,CONCEICAO DA BARRA,Ultrapassagem Indevida,Colisão lateral sentido oposto,⋯,1,1,1,1,3,NA,NA,SPRF-ES,DEL04-ES,UOP02-DEL04-ES
2,571804,2024-01-01,segunda-feira,17400 secs,PI,343,185,PIRIPIRI,Manobra de mudança de faixa,Colisão frontal,⋯,0,1,0,0,2,NA,NA,SPRF-PI,DEL02-PI,UOP01-DEL02-PI
3,571806,2024-01-01,segunda-feira,16200 secs,BA,116,578,BREJOES,Ingestão de álcool pelo condutor,Colisão frontal,⋯,0,1,2,0,4,NA,NA,SPRF-BA,DEL03-BA,UOP02-DEL03-BA
4,571818,2024-01-01,segunda-feira,23400 secs,SE,101,18,MALHADA DOS BOIS,Reação tardia ou ineficiente do condutor,Saída de leito carroçável,⋯,1,0,2,1,3,NA,NA,SPRF-SE,DEL02-SE,UOP02-DEL02-SE
5,571838,2024-01-01,segunda-feira,18000 secs,MT,364,240,RONDONOPOLIS,Condutor deixou de manter distância do veículo da frente,Colisão traseira,⋯,0,2,1,0,3,NA,NA,SPRF-MT,DEL02-MT,UOP01-DEL02-MT
6,571855,2024-01-01,segunda-feira,42600 secs,MG,251,447,GRAO MOGOL,Velocidade Incompatível,Colisão traseira,⋯,0,1,1,1,3,NA,NA,SPRF-MG,DEL12-MG,UOP01-DEL12-MG


,UF,Rodovia,km inicial,km final,Extensão km,Data,Latitude,Longitude,Observação,ICC,ICP,ICM
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1,PE,BR-232,324.4,325,0.6,2021-08-02,-8.091913,-37.57831,NA,100,12.5,38.75
2,PE,BR-232,325.0,326,1.0,2021-08-02,-8.091703,-37.58753,NA,100,0.0,30.00
3,PE,BR-232,326.0,327,1.0,2021-08-02,-8.091090,-37.59603,Ausencia de Sinalizacao Vertical,90,0.0,27.00
4,PE,BR-232,327.0,328,1.0,2021-08-02,-8.090370,-37.60506,NA,75,0.0,22.50
5,PE,BR-232,328.0,329,1.0,2021-08-02,-8.089709,-37.61390,NA,65,0.0,19.50
6,PE,BR-232,329.0,330,1.0,2021-08-02,-8.089044,-37.62308,Inicio do Trecho Urbano Custodia,65,0.0,19.50


,ID,vl_codigo,UF,REGIAO,RELEVO_PRE,VELOCIDADE,CODIGO_BR,EXTENSAO,SUPERFICIE,JURISDICAO,TIPO_LINK,CLASSIFICACAO,GEH,VMDA_C,VMDA_D,VMDA_TOTAL,CODIGO_SNV.SRE,Ano
,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
1,4317,307BAC0005,AC,NA,NA,NA,307,804,PLA,FEDERAL-PLA,Eixo Principal,Planejada,NA,NA,NA,NA,NA,2020
2,4318,307BAC0010,AC,NA,NA,NA,307,356,PLA,FEDERAL-PLA,Eixo Principal,Planejada,NA,NA,NA,NA,NA,2020
3,4319,307BAC0015,AC,NA,NA,NA,307,57,PLA,FEDERAL-PLA,Eixo Principal,Planejada,NA,NA,NA,NA,NA,2020
4,4320,307BAC0018,AC,NA,NA,NA,307,95,PLA,FEDERAL-PLA,Eixo Principal,Planejada,NA,NA,NA,NA,NA,2020
5,4321,307BAC0020,AC,NA,NA,NA,307,254,PLA,NA,Eixo Principal,Não Simulável,NA,NA,NA,NA,NA,2020
6,4322,307BAC0022,AC,NA,NA,NA,307,224,PAV,FEDERAL-PAV,Eixo Principal,NA,NA,833,871,1704,NA,2020


,BR,UF,Tipo de trecho,Desc Coinc,Código,Local de Início,Local de Fim,km inicial,km final,Extensão,Superfície Federal,OBRAS,Federal Coincidente,Administração,Ato legal,Estadual Coincidente,Superfície Est. Coincidente,Jurisdição,Superfície,Unidade Local
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,10,DF,Eixo Principal,NA,010BDF0010,ENTR BR-020(A)/030(A)/450/DF-001 (BRASÍLIA),ENTR DF-440,0.0,2.4,2.4,DUP,NA,030BDF0010;020BDF0010,Convênio de Administração,NA,NA,NA,Federal,PAV,Brasília
2,10,DF,Eixo Principal,NA,010BDF0015,ENTR DF-440,ACESSO I SOBRADINHO,2.4,6.0,3.6,DUP,NA,030BDF0015;020BDF0015,Convênio de Administração,NA,NA,NA,Federal,PAV,Brasília
3,10,DF,Eixo Principal,NA,010BDF0016,ACESSO I SOBRADINHO,ACESSO II SOBRADINHO,6.0,8.3,2.3,DUP,NA,030BDF0016;020BDF0016,Convênio de Administração,NA,NA,NA,Federal,PAV,Brasília
4,10,DF,Eixo Principal,NA,010BDF0018,ACESSO II SOBRADINHO,ENTR DF-230,8.3,18.2,9.9,DUP,NA,030BDF0018;020BDF0018,Convênio de Administração,NA,NA,NA,Federal,PAV,Brasília
5,10,DF,Eixo Principal,NA,010BDF0020,ENTR DF-230,ENTR DF-128,18.2,22.0,3.8,DUP,NA,030BDF0020;020BDF0020,Convênio de Administração,NA,NA,NA,Federal,PAV,Brasília
6,10,DF,Eixo Principal,NA,010BDF0022,ENTR DF-128,P/PLANALTINA,22.0,25.0,3.0,DUP,NA,030BDF0022;020BDF0022,Convênio de Administração,NA,NA,NA,Federal,PAV,Brasília


Análise das Variáveis de Conexão
Com base na estrutura típica desses dados (PRF, tráfego e SNV), as variáveis-chave para união são:

1. Identificador de Rodovia (BR)
DataFrame	Variável Provável
df_acidentes_prf	br, br_numero
df_trafego	br, rodovia
df_snv	br, sg_br
icm_mes_a_mes	br

2. Quilometragem (KM)
DataFrame	Variável Provável
df_acidentes_prf	km
df_trafego	km_inicial, km_final
df_snv	km_inicial, km_final

3. UF (Unidade Federativa)
DataFrame	Variável Provável
df_acidentes_prf	uf
df_trafego	uf, sg_uf
df_snv	uf, sg_uf
icm_mes_a_mes	uf

4. Município
DataFrame	Variável Provável
df_acidentes_prf	municipio, cod_municipio
df_snv	nm_municipio, cd_municipio

5. Período Temporal
DataFrame	Variável Provável
df_acidentes_prf	data_inversa, ano, mes
icm_mes_a_mes	ano, mes, data


Resumo das Conexões

Conexão	Chave Principal	Tipo de Relação
Acidentes ↔ SNV	BR + KM + UF	Localização do trecho
Acidentes ↔ Tráfego	BR + UF	Volume no trecho
Acidentes ↔ ICM	UF + Ano + Mês	Índice econômico temporal

Nota: A variável km em acidentes é pontual, enquanto no SNV e tráfego geralmente é uma faixa (km_inicial a km_final). Será necessário criar uma lógica de range join ou arredondar/discretizar o km.

In [ ]:
# NÃO FICOU BOM

# Exemplo de união dos dataframes

# 1. Unir acidentes com SNV (localização geográfica)
# Chaves: BR + KM (aproximado) + UF
df_acidentes_snv = df_acidentes_prf.merge(
    df_snv,
    left_on=['br', 'uf'],
    right_on=['sg_br', 'sg_uf'],
    how='left'
)

# 2. Unir com tráfego (volume de veículos)
# Chaves: BR + UF + faixa de KM
df_completo = df_acidentes_snv.merge(
    df_trafego,
    on=['br', 'uf'],
    how='left'
)

# 3. Unir com ICM (índices mensais)
# Chaves: UF + ano + mês
df_final = df_completo.merge(
    icm_mes_a_mes,
    on=['uf', 'ano', 'mes'],
    how='left'
)

In [8]:
ls()

[1] "df_acidentes_prf" "df_snv"           "df_trafego"       "icm_mes_a_mes"

In [11]:
# Exportar os dataframes em individual formato CSV para carregar como tabela no clickhouse

write.csv(df_acidentes_prf, "Trabalho/pre-processamentos/df_acidentes_prf.csv", row.names = FALSE, na = "")

write.csv(icm_mes_a_mes, "Trabalho/pre-processamentos/icm_mes_a_mes.csv", row.names = FALSE, na = "")

write.csv(df_trafego, "Trabalho/pre-processamentos/df_trafego.csv", row.names = FALSE, na = "")

write.csv(df_snv, "Trabalho/pre-processamentos/df_snv.csv", row.names = FALSE, na = "")

Criar tabela no DB

```sql
CREATE TABLE IF NOT EXISTS prf_acidentes (
    id UInt32,
    data_inversa Date,
    dia_semana String,
    horario String, -- formato HH:MM:SS
    uf FixedString(2),
    br UInt16,
    km Float64,
    municipio String,
    causa_acidente String,
    tipo_acidente String,
    classificacao_acidente String,
    fase_dia String,
    sentido_via String,
    condicao_metereologica String,
    tipo_pista String,
    tracado_via String,
    uso_solo String,
    pessoas UInt16,
    mortos UInt16,
    feridos_leves UInt16,
    feridos_graves UInt16,
    ilesos UInt16,
    ignorados UInt16,
    feridos UInt16,
    veiculos UInt16,
    latitude Nullable(Float64),
    longitude Nullable(Float64),
    regional String,
    delegacia String,
    uop String
)
ENGINE = MergeTree
ORDER BY id;

drop TABLE IF EXISTS icm_mes_a_mes;
CREATE TABLE IF NOT EXISTS icm_mes_a_mes (
    uf FixedString(20),
    rodovia String,              -- ex: BR-163
    km_inicial Float64,
    km_final Float64,
    extensao_km Float64,
    data Date,                   -- formato YYYY-MM-DD
    latitude Nullable(Float64),
    longitude Nullable(Float64),
    observacao String,
    icc Float64,
    icp Float64,
    icm Float64
)
ENGINE = MergeTree
ORDER BY (uf, rodovia, data, km_inicial);

drop TABLE IF EXISTS df_trafego;
CREATE TABLE IF NOT EXISTS df_trafego (
    ID UInt32,
    vl_codigo String,
    UF FixedString(2),
    REGIAO String,
    RELEVO_PRE String,
    VELOCIDADE Nullable(UInt16),
    CODIGO_BR UInt16,
    EXTENSAO Float64,
    SUPERFICIE String,
    JURISDICAO String,
    TIPO_LINK String,
    CLASSIFICACAO String,
    GEH Nullable(FixedString(10)),
    VMDA_C Nullable(UInt32),
    VMDA_D Nullable(UInt32),
    VMDA_TOTAL Nullable(UInt32),
    `CODIGO_SNV.SRE` String,
    Ano UInt16
)
ENGINE = MergeTree
ORDER BY (UF, CODIGO_BR, ID);

CREATE TABLE IF NOT EXISTS df_snv (
    BR UInt16,
    UF FixedString(2),
    Tipo_de_trecho String,
    Desc_Coinc String,
    Codigo String,
    Local_de_Inicio String,
    Local_de_Fim String,
    km_inicial Float64,
    km_final Float64,
    Extensao Float64,
    Superficie_Federal String,
    OBRAS String,
    Federal_Coincidente String,
    Administracao String,
    Ato_legal String,
    Estadual_Coincidente String,
    Superficie_Est_Coincidente String,
    Jurisdicao String,
    Superficie String,
    Unidade_Local String
)
ENGINE = MergeTree
ORDER BY (UF, BR, km_inicial);
```

### Remover o cabeçalho e importar

```bash
clickhouse-client --query="INSERT INTO prf_acidentes FORMAT CSV" --input_format_skip_unknown_fields=1 < <(tail -n +2 df_acidentes_prf.csv)

clickhouse-client --query="INSERT INTO icm_mes_a_mes FORMAT CSV" --input_format_skip_unknown_fields=1 < <(tail -n +2 icm_mes_a_mes.csv)

clickhouse-client --query="INSERT INTO df_trafego FORMAT CSV" --input_format_skip_unknown_fields=1 < <(tail -n +2 df_trafego.csv)

clickhouse-client --query="INSERT INTO df_snv FORMAT CSV" --input_format_skip_unknown_fields=1 < <(tail -n +2 df_snv.csv)
```


Agora, vamos escolher os atributos

## prf_acidentes 
    data_inversa,
    horario,
    uf,
    br,
    km,
    municipio,
    causa_acidente,
    tipo_acidente,
    classificacao_acidente,
    fase_dia,
    sentido_via,
    condicao_metereologica,
    tipo_pista,
    tracado_via,
    uso_solo,
    pessoas,
    mortos,
    feridos_leves,
    feridos_graves,
    ilesos,
    ignorados,
    feridos,
    veiculos,
    latitude,
    longitude

## icm_mes_a_mes
    uf, 
    rodovia,
    km_inicial,
    km_final,
    extensao_km,
    data,
    latitude,
    longitude,
    observacao,
    icc,
    icp,
    icm

## df_trafego
    vl_codigo,
    UF,
    REGIAO,
    RELEVO_PRE,
    VELOCIDADE,
    CODIGO_BR,
    EXTENSAO,
    SUPERFICIE,
    JURISDICAO,
    TIPO_LINK,
    CLASSIFICACAO,
    GEH,
    VMDA_C,
    VMDA_D,
    VMDA_TOTAL,
    Ano

## df_snv
    BR,
    UF,
    Tipo_de_trecho,
    Desc_Coinc,
    Codigo,
    Local_de_Inicio,
    Local_de_Fim,
    km_inicial,
    km_final,
    Extensao,
    Superficie_Federal,
    OBRAS,
    Federal_Coincidente,
    Administracao,
    Ato_legal,
    Estadual_Coincidente,
    Superficie_Est_Coincidente,
    Jurisdicao,
    Superficie,
    Unidade_Local


## Liga prf_acidentes com df_snv via (br, km, uf)

```sql
select count(*) from prf_acidentes;
```
73156

```sql
select count(*) from df_snv;
```
7297



```sql
SELECT 
       a.uf, -- chave entre acidentes e snv --& icm
       a.br, -- chave entre acidentes e snv --& icm
       a.km, -- chave entre acidentes e snv
       a.data_inversa,
       a.horario,
       a.municipio,
       a.causa_acidente,
       a.tipo_acidente,
       a.classificacao_acidente,
       a.fase_dia,
       a.sentido_via,
       a.condicao_metereologica,
       a.tipo_pista,
       a.tracado_via,
       a.uso_solo,
       a.pessoas,
       a.mortos,
       a.feridos_leves,
       a.feridos_graves,
       a.ilesos,
       a.ignorados,
       a.feridos,
       a.veiculos,
       a.latitude,
       a.longitude,
       b.Codigo  -- chave para trafego
FROM prf_acidentes a
INNER JOIN df_snv b
ON a.br = b.BR
AND a.uf = b.UF
AND a.km BETWEEN b.km_inicial AND b.km_final;
```

83054 rows in set.

Vinculo entre df_snv (codigo) e df_trafego (vl_codigo)

```sql
select count(*) from df_trafego;
```
29817

```sql
select count(*) from df_snv;
```
7297


```sql
SELECT 
    a.vl_codigo,
    a.UF,
    a.REGIAO,
    a.RELEVO_PRE,
    a.VELOCIDADE,
    a.CODIGO_BR,
    a.EXTENSAO,
    a.SUPERFICIE,
    a.JURISDICAO,
    a.TIPO_LINK,
    a.CLASSIFICACAO,
    a.GEH,
    a.VMDA_C,
    a.VMDA_D,
    a.VMDA_TOTAL,
    a.Ano,
    b.UF,
    b.BR
FROM df_trafego a
INNER JOIN df_snv b
ON a.vl_codigo = b.Codigo;
```

6729 rows in set.

## junta com icm

agora junta com icm (rodovia)

```sql
select count(*) from icm_mes_a_mes;
```
394230

```sql
SELECT 
    a.vl_codigo,
    a.UF,
    a.REGIAO,
    a.RELEVO_PRE,
    a.VELOCIDADE,
    a.CODIGO_BR,
    a.EXTENSAO,
    a.SUPERFICIE,
    a.JURISDICAO,
    a.TIPO_LINK,
    a.CLASSIFICACAO,
    a.GEH,
    a.VMDA_C,
    a.VMDA_D,
    a.VMDA_TOTAL,
    a.Ano,
    b.UF,
    b.BR
FROM df_trafego a
INNER JOIN df_snv b
ON a.vl_codigo = b.Codigo
INNER JOIN icm_mes_a_mes c
ON concat(b.UF, '-', toString(b.BR)) = c.rodovia;
```
0

Não consegui juntar, pois sobre o ICM temos apenas as BRs:

  ┌─count()─┬─rodovia─┐
  1. │    2145 │ BR-110  │
  2. │     695 │ RJ      │
  3. │     426 │ BR-386  │
  4. │    1928 │ BR-359  │
  5. │    9702 │ BR-116  │
  6. │     249 │ BR-420  │
  7. │     106 │ BR-485  │
  8. │    2031 │ BR-422  │
  9. │    1005 │ BR-487  │
 10. │     926 │ BR-030  │
 11. │    7867 │ BR-262  │
 12. │     110 │ BR-426  │
 13. │    1087 │ BR-424  │
 14. │      45 │ BR-498  │
 15. │    1177 │ BR-425  │
 16. │    2336 │ PA      │
 17. │     211 │ BR-480  │
 18. │     839 │ BR-427  │
 19. │     798 │ AM      │
 20. │     264 │ BR-459  │
 21. │    6693 │ BR-267  │
 22. │    2561 │ BR-423  │
 23. │     490 │ BR-421  │
 24. │    5857 │ BR-343  │
 25. │    3472 │ MA      │
 26. │   15633 │ BR-174  │
 27. │     597 │ BR-381  │
 28. │     306 │ BR-324  │
 29. │     120 │ BR-434  │
 30. │      94 │ BR-488  │
 31. │    1724 │ BR-493  │
 32. │     672 │ BR-272  │
 33. │     180 │ BR-436  │
 34. │    1638 │ BR-432  │
 35. │    9448 │ BR-020  │
 36. │    1008 │ AL      │
 37. │     422 │ BR-430  │
 38. │     336 │ BR-495  │
 39. │    1296 │ BR-429  │
 40. │    3873 │ BR-104  │
 41. │     501 │ BR-349  │
 42. │     264 │ BR-354  │
 43. │     651 │ BR-392  │
 44. │    1571 │ SC      │
 45. │   15282 │ BR-163  │
 46. │       2 │ Rodovia │
 47. │     579 │ AP      │
 48. │    1701 │ BR-356  │
 49. │   15983 │ BR-101  │
 50. │      49 │ BR-393  │
 51. │    1372 │ BR-428  │
 52. │    1188 │ BR-452  │
 53. │     174 │ BR-040  │
 54. │    2029 │ BR-277  │
 55. │    1955 │ BR-080  │
 56. │    1886 │ BR-210  │
 57. │     860 │ BR-435  │
 58. │     358 │ SE      │
 59. │     902 │ BR-414  │
 60. │     420 │ BR-416  │
 61. │    3199 │ BR-235  │
 62. │     645 │ BR-412  │
 63. │    2805 │ PI      │
 64. │    1559 │ BR-290  │
 65. │     572 │ BR-468  │
 66. │    1572 │ PB      │
 67. │    1816 │ RO      │
 68. │      36 │ BR-410  │
 69. │     864 │ BR-308  │
 70. │     964 │ BR-369  │
 71. │    2796 │ BR-317  │
 72. │    1823 │ BR-122  │
 73. │   17803 │ BR-158  │
 74. │    3553 │ BR-376  │
 75. │    1920 │ TO      │
 76. │    1180 │ AC      │
 77. │     114 │ SP      │
 78. │    2267 │ CE      │
 79. │   16791 │ BR-316  │
 80. │    3057 │ MS      │
 81. │     798 │ BR-373  │
 82. │    1152 │ BR-408  │
 83. │     996 │ BR-293  │
 84. │    1299 │ BR-472  │
 85. │    9247 │ BR-060  │
 86. │     118 │ BR-469  │
 87. │    4743 │ BR-470  │
 88. │   15152 │ BR-230  │
 89. │    1547 │ BR-251  │
 90. │      62 │ BR-415  │
 91. │    2218 │ BR-476  │
 92. │    3069 │ BR-232  │
 93. │   14843 │ BR-153  │
 94. │       1 │ UF      │
 95. │      61 │ BR-307  │
 96. │      28 │ ES      │
 97. │   27096 │ BR-364  │
 98. │    2358 │ PE      │
 99. │    1904 │ BR-155  │
100. │    1175 │ BR-463  │
101. │    1017 │ BR-419  │
102. │    7051 │ BR-282  │
103. │    3514 │ BR-402  │
104. │    3304 │ BR-280  │
105. │    9097 │ BR-010  │
106. │     688 │ BR-406  │
107. │    6205 │ BR-242  │
108. │     218 │ BR-465  │
109. │     800 │ BR-259  │
110. │      96 │ BR-467  │
111. │    1546 │ PR      │
112. │     380 │ BR-404  │
113. │   12064 │ BR-222  │
114. │     957 │ BR-405  │
115. │     891 │ BR-287  │
116. │    2978 │ BR-407  │
117. │    2858 │ BR-285  │
118. │     587 │ BR-403  │
119. │     378 │ BR-418  │
120. │    3832 │ MT      │
121. │    9077 │ BR-226  │
122. │     798 │ BR-401  │
123. │    9472 │ BR-070  │
124. │    3757 │ BR-319  │
125. │   11669 │ BR-135  │
126. │    3653 │ BR-156  │
127. │     565 │ BR-361  │
128. │    1153 │ RR      │
129. │     488 │ BR-365  │
130. │    1491 │ BR-367  │
131. │    2349 │ BR-304  │
     └─count()─┴─rodovia─┘

